<a href="https://colab.research.google.com/github/zackives/upenn-cis-2450/blob/main/Module_1_Data_Acquisition_Wrangling_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview of Part I of Big Data Analytics

As we start our journey into Big Data Analytics, the first thing we need to do is **get the data** in the form we need for analysis!  We'll start with an overview of how to acquire and *wrangle* data.

This notebook will be built incrementally to consider several tasks:

* Acquiring data from files and remote sources
* Information extraction over HTML content
* A basic "vocabulary" of operators over tables (the relational algebra)
* Basic manipulation using SQL in DuckDB

* "Data wrangling" or integration:
  * Cleaning and filtering data, using rules and based operations
  * Linking data across dataframes or relations
  * The need for approximate match and record linking
  * Different techniques


## The Motivating Question
To illustrate the principles, we focus on the question of **how old company CEOs and founders** (in general, leaders) are.  The question was in part motivated by the following New York Times article:

* Founders of Successful Tech Companies Are Mostly Middle-Aged: https://www.nytimes.com/2019/08/29/business/tech-start-up-founders-nest.html?searchResultPosition=2

So let's test this hypothesis!

## Initial Libraries

We'll be using [DuckDB](https://duckdb.org/) as a means of managing our tables.  DuckDB works like a Python library, but manages a full SQL database (in files).  It also integrates very nicely with Pandas, so we'll use it in this course.

In [ ]:
!pip3 install duckdb

In [ ]:
!pip3 install lxml

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [ ]:
!pip3 install penngrader-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00


For quiz credit you'll need to update your student ID here!

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

Quizzes will cumulatively count as HW9... Don't edit this...

In [ ]:
%set_env HW_ID=cis2450_fall24_HW9

env: HW_ID=cis2450_fall24_HW9


In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


In [ ]:
# Imports we'll use through the notebook, collected here for simplicity

# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib
import urllib.request

# Pandas dataframes and operations
import pandas as pd

# Numpy matrix and array operations
import numpy as np

# Sqlite is a simplistic database
import duckdb

# Data visualization
import matplotlib



# 1. Acquiring External Data

To test our hypothesis, we might want:

1. A list of companies (and, for futher details, perhaps their lines of business)
2. A list of company CEOs
3. Ages of the CEOs

We'll go through each of these using real data from the web.

### Reading Structured Data Sources

Let's start by looking up data about companies.  We are using a dataset from: https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset?resource=download

but we have a copy of it at an alternate site for convenience of downloading.

## 1.1. External CSV Data

Comma-separated values are generally easy to read. The main questions are column headings (which are in an optional row that isn't always provided) and datatypes (which might default to the wrong thing).

In [ ]:
!wget -nc https://storage.googleapis.com/penn-cis5450/companies_sorted.csv

--2024-08-26 22:45:18--  https://storage.googleapis.com/penn-cis5450/companies_sorted.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1085578742 (1.0G) [text/csv]
Saving to: ‘companies_sorted.csv’

companies_sorted.cs 100%[===================>]   1.01G  51.7MB/s    in 18s     

2024-08-26 22:45:36 (57.3 MB/s) - ‘companies_sorted.csv’ saved [1085578742/1085578742]



In [ ]:
# This reads remotely. To avoid multiple fetches, we'll instead..

# data = urllib.request.urlopen(\
#        'https://storage.googleapis.com/penn-cis5450/companies_sorted.csv')
# company_data_df = pd.read_csv(data)

## ... instead copy to a local file and read there...

company_data_df = pd.read_csv('companies_sorted.csv')

company_data_df

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
7173421,1494427,certiport vouchers,certiportvouchers.com,2011.0,information technology and services,1 - 10,NaN,NaN,linkedin.com/company/certiport-vouchers,0,1
7173422,1494429,black tiger fight club,blacktigerclub.com,2006.0,"health, wellness and fitness",1 - 10,"peking, beijing, china",china,linkedin.com/company/black-tiger-club-hero,0,6
7173423,4768462,catholic bishop of chicago,NaN,NaN,religious institutions,1 - 10,"inverness, illinois, united states",united states,linkedin.com/company/catholic-bishop-of-chicago,0,1
7173424,1494436,medexo robotics ltd,NaN,NaN,research,1 - 10,"london, london, united kingdom",united kingdom,linkedin.com/company/medexo-robotics-ltd,0,2


Now let's use DuckDB to work with the dataframe.

In [ ]:
# We can ask for the contents of a Pandas Dataframe through DuckDB, in the SQL language.
duckdb.sql('select * from company_data_df')

┌────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ Unnamed: 0 │         name         │ … │     linkedin url     │ current employee e…  │ total employee est…  │
│   int64    │       varchar        │   │       varchar        │        int64         │        int64         │
├────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│    5872184 │ ibm                  │ … │ linkedin.com/compa…  │               274047 │               716906 │
│    4425416 │ tata consultancy s…  │ … │ linkedin.com/compa…  │               190771 │               341369 │
│      21074 │ accenture            │ … │ linkedin.com/compa…  │               190689 │               455768 │
│    2309813 │ us army              │ … │ linkedin.com/compa…  │               162163 │               445958 │
│    1558607 │ ey                   │ … │ linkedin.com/compa…  │               158363 │               428960 │
│

## 1.2. Storing Data Locally and Re-Loading it

DuckDB nicely integrates with Pandas and Python. If you create a connection to a file, this results in the creation of a database stored within that file.

Normally we need to `CREATE TABLE` with the table name and columns. But we can actually create the table to match the *schema* of the DataFrame, as follows.

In [ ]:
con = duckdb.connect('local.db')
con.sql('create table if not exists company_data as select * from company_data_df')

# query the table
con.table("company_data").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ Unnamed: 0 │         name         │ … │     linkedin url     │ current employee e…  │ total employee est…  │
│   int64    │       varchar        │   │       varchar        │        int64         │        int64         │
├────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│    5872184 │ ibm                  │ … │ linkedin.com/compa…  │               274047 │               716906 │
│    4425416 │ tata consultancy s…  │ … │ linkedin.com/compa…  │               190771 │               341369 │
│      21074 │ accenture            │ … │ linkedin.com/compa…  │               190689 │               455768 │
│    2309813 │ us army              │ … │ linkedin.com/compa…  │               162163 │               445958 │
│    1558607 │ ey                   │ … │ linkedin.com/compa…  │               158363 │               428960 │
│

In [ ]:
company_data_df = con.table("company_data").df()

company_data_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
7173421,1494427,certiport vouchers,certiportvouchers.com,2011.0,information technology and services,1 - 10,None,None,linkedin.com/company/certiport-vouchers,0,1
7173422,1494429,black tiger fight club,blacktigerclub.com,2006.0,"health, wellness and fitness",1 - 10,"peking, beijing, china",china,linkedin.com/company/black-tiger-club-hero,0,6
7173423,4768462,catholic bishop of chicago,None,NaN,religious institutions,1 - 10,"inverness, illinois, united states",united states,linkedin.com/company/catholic-bishop-of-chicago,0,1
7173424,1494436,medexo robotics ltd,None,NaN,research,1 - 10,"london, london, united kingdom",united kingdom,linkedin.com/company/medexo-robotics-ltd,0,2


## 1.3. Companies' CEOs: a Web Table

Now we need to figure out who the CEOs are for corporations.  One place to look is Wikipedia, which has an HTML table describing the CEOs.

https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs

Pandas actually makes it easy to read HTML tables...

In [ ]:
# Now let's read an HTML table!

company_ceos_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs')[1]

company_ceos_df

,Company,Executive,Title,Since,Notes,Updated
0,Accenture,Julie Sweet,CEO[1],2019,"Succeeded Pierre Nanterme, died",2019-01-31
1,Aditya Birla Group,Kumar Mangalam Birla,Chairman[2],1995[2],Part of the Birla family business house in India,2018-10-01
2,Adobe Systems,Shantanu Narayen,"Chairman, president and CEO[3]",2007,Formerly with Apple,2018-10-01
3,Airbus,Guillaume Faury,CEO[4],2012,Succeeded Louis Gallois,2017-11-14
4,Alibaba,Eddie Wu,Director and CEO[5],2023[6],NaN,2024-07-19
...,...,...,...,...,...,...
130,Warner Brothers,Ann Sarnoff,Chairwoman and CEO[122],2019,First woman to hold the position at the compan...,2019-10-10
131,WarnerMedia,Jason Kilar,CEO[123],2020,Previously with Hulu and Amazon,2020-11-19
132,Wells Fargo,Charles Scharf,CEO and president[124],2019,"Succeeded John Stumpf, previously COO",NaN
133,Whole Foods Market,John Mackey,CEO[125],1980,Co-founder,2017-11-11


In [ ]:
con.sql('create table if not exists company_ceos as select * from company_ceos_df')

con.table('company_ceos').show()

┌──────────────────────┬──────────────────────┬──────────────────────┬─────────┬──────────────────────────┬────────────┐
│       Company        │      Executive       │        Title         │  Since  │          Notes           │  Updated   │
│       varchar        │       varchar        │       varchar        │ varchar │         varchar          │  varchar   │
├──────────────────────┼──────────────────────┼──────────────────────┼─────────┼──────────────────────────┼────────────┤
│ Accenture            │ Julie Sweet          │ CEO[1]               │ 2019    │ Succeeded Pierre Nante…  │ 2019-01-31 │
│ Aditya Birla Group   │ Kumar Mangalam Birla │ Chairman[2]          │ 1995[2] │ Part of the Birla fami…  │ 2018-10-01 │
│ Adobe Systems        │ Shantanu Narayen     │ Chairman, presiden…  │ 2007    │ Formerly with Apple      │ 2018-10-01 │
│ Airbus               │ Guillaume Faury      │ CEO[4]               │ 2012    │ Succeeded Louis Gallois  │ 2017-11-14 │
│ Alibaba              │ Eddie W

## 1.4. The Problem Gets Harder... Extracting Fields from Tagged Data on the Web

So far we have companies and CEOs.  But we don't have information on how old the CEOs are!

For a solution, we're going to go back to Wikipedia -- this time looking at the web pages for the CEOs!

This involves "crawling" the CEO pages, and "scraping" the relevant content.  In other words we have to do *information extraction*.  For this particular problem, we will do extraction over very regular parts of Wikipedia.

We'll start by constructing a list of CEO web pages, from the Company CEO dataframe above.  For this, we need to take the names and do a bit of tweaking, for example adding underscores instead of spaces.

(Later we'll see how to do this over more free-form text.)

In [ ]:
crawl_list = []

for executive in company_ceos_df['Executive']:
  crawl_list.append('https://en.wikipedia.org/wiki/' + executive.replace(' ', '_'))

crawl_list

['https://en.wikipedia.org/wiki/Julie_Sweet',
 'https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla',
 'https://en.wikipedia.org/wiki/Shantanu_Narayen',
 'https://en.wikipedia.org/wiki/Guillaume_Faury',
 'https://en.wikipedia.org/wiki/Eddie_Wu',
 'https://en.wikipedia.org/wiki/Andy_Jassy',
 'https://en.wikipedia.org/wiki/Lisa_Su',
 'https://en.wikipedia.org/wiki/Stephen_Squeri',
 'https://en.wikipedia.org/wiki/Joseph_R._Swedish',
 'https://en.wikipedia.org/wiki/Tim_Cook',
 'https://en.wikipedia.org/wiki/Lakshmi_Niwas_Mittal',
 'https://en.wikipedia.org/wiki/John_Stankey',
 'https://en.wikipedia.org/wiki/Charles_Woodburn',
 'https://en.wikipedia.org/wiki/Tapan_Singhel',
 'https://en.wikipedia.org/wiki/Carlos_Torres_Vila',
 'https://en.wikipedia.org/wiki/Brian_Moynihan',
 'https://en.wikipedia.org/wiki/C.S._Venkatakrishnan',
 'https://en.wikipedia.org/wiki/Warren_Buffett',
 'https://en.wikipedia.org/wiki/Hubert_Joly',
 'https://en.wikipedia.org/wiki/Sunil_Bharti_Mittal',
 'https://en.wiki

In [ ]:
# Use urllib.urlopen to crawl all pages in crawl_list, and store the response of the page
# in list pages

pages = []

for url in crawl_list:
    page = url.split("/")[-1] #extract the person name at the end of the url

    # An issue: some of the accent characters won't work.  We need to convert them
    # into an HTML URL.  We'll split the URL, then use "parse.quote" to change
    # the structure, then re-form the URL
    url_list = list(urllib.parse.urlsplit(url))
    url_list[2] = urllib.parse.quote(url_list[2])
    url_ascii = urllib.parse.urlunsplit(url_list)
    try:
      response = urllib.request.urlopen((url_ascii))
      #Save page and url for later use.
      pages.append(response)
    except urllib.error.URLError as e:
      print(e.reason)


## 1.5. Crawling: Populating the Table with Executives

In [ ]:
# Use lxml.etree.HTML(...) on the HTML content of each page to get a DOM tree that
# can be processed via XPath to extract the bday information.  Store the CEO name,
# webpage, and the birthdate (born) in exec_df.

# We first check that the HTML content has a table of type `vcard`,
# and then extract the `bday` information.  If there is no birthdate, the datetime
# value is NaT (not a type).

from lxml import etree

rows = []
for page in pages:
  url = page.geturl()
  print (url)
  content = page.read().decode("utf-8")
  tree = etree.HTML(content)  #create a DOM tree of the page
  bday = tree.xpath('//table[contains(@class,"vcard")]//span[@class="bday"]/text()')
  if len(bday) > 0:
      name = url[url.rfind('/')+1:] # The part of the URL after the last /
      rows.append({'name': name, 'page': url,
                  'born': datetime.datetime.strptime(bday[0], '%Y-%m-%d')})
  else:
          rows.append({'name': url[url.rfind('/')+1:], 'page': url
                                    , 'born': np.datetime64('NaT')})

exec_df = pd.DataFrame(rows)
exec_df

https://en.wikipedia.org/wiki/Julie_Sweet
https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla
https://en.wikipedia.org/wiki/Shantanu_Narayen
https://en.wikipedia.org/wiki/Guillaume_Faury
https://en.wikipedia.org/wiki/Eddie_Wu
https://en.wikipedia.org/wiki/Andy_Jassy
https://en.wikipedia.org/wiki/Lisa_Su
https://en.wikipedia.org/wiki/Stephen_Squeri
https://en.wikipedia.org/wiki/Joseph_R._Swedish
https://en.wikipedia.org/wiki/Tim_Cook
https://en.wikipedia.org/wiki/Lakshmi_Niwas_Mittal
https://en.wikipedia.org/wiki/John_Stankey
https://en.wikipedia.org/wiki/Charles_Woodburn
https://en.wikipedia.org/wiki/Tapan_Singhel
https://en.wikipedia.org/wiki/Carlos_Torres_Vila
https://en.wikipedia.org/wiki/Brian_Moynihan
https://en.wikipedia.org/wiki/C.S._Venkatakrishnan
https://en.wikipedia.org/wiki/Warren_Buffett
https://en.wikipedia.org/wiki/Hubert_Joly
https://en.wikipedia.org/wiki/Sunil_Bharti_Mittal
https://en.wikipedia.org/wiki/Stephen_A._Schwarzman
https://en.wikipedia.org/wiki/Mike_Henry
http

,name,page,born
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_B...,1967-06-14 00:00:00
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00
4,Eddie_Wu,https://en.wikipedia.org/wiki/Eddie_Wu,NaT
...,...,...,...
130,Ann_Sarnoff,https://en.wikipedia.org/wiki/Ann_Sarnoff,NaT
131,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00
132,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24 00:00:00
133,John_Mackey,https://en.wikipedia.org/wiki/John_Mackey,NaT


In [ ]:
con.sql('create table if not exists executives as select * from exec_df')

con.table('executives').show()

┌──────────────────────┬────────────────────────────────────────────────────┬─────────────────────┐
│         name         │                        page                        │        born         │
│       varchar        │                      varchar                       │      timestamp      │
├──────────────────────┼────────────────────────────────────────────────────┼─────────────────────┤
│ Julie_Sweet          │ https://en.wikipedia.org/wiki/Julie_Sweet          │ NULL                │
│ Kumar_Mangalam_Birla │ https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla │ 1967-06-14 00:00:00 │
│ Shantanu_Narayen     │ https://en.wikipedia.org/wiki/Shantanu_Narayen     │ 1963-05-27 00:00:00 │
│ Guillaume_Faury      │ https://en.wikipedia.org/wiki/Guillaume_Faury      │ 1968-02-22 00:00:00 │
│ Eddie_Wu             │ https://en.wikipedia.org/wiki/Eddie_Wu             │ NULL                │
│ Andy_Jassy           │ https://en.wikipedia.org/wiki/Andy_Jassy           │ 1968-01-13 00:00:00 │


# 2.0 Data Transformation and Querying

Looking at our data to clean via *projection*...

Generally, we can extract one "narrower" table form another by using **double brackets**.

In [ ]:
# Let's take a look at the data.  Here's a way of PROJECTING the exec_df dataframe into
# a smaller table

exec_df[['name', 'born']]

,name,born
0,Julie_Sweet,NaT
1,Kumar_Mangalam_Birla,1967-06-14 00:00:00
2,Shantanu_Narayen,1963-05-27 00:00:00
3,Guillaume_Faury,1968-02-22 00:00:00
4,Eddie_Wu,NaT
...,...,...
130,Ann_Sarnoff,NaT
131,Jason_Kilar,1971-04-26 00:00:00
132,Charles_Scharf,1965-04-24 00:00:00
133,John_Mackey,NaT


In [ ]:
# In SQL it's SELECT with the fields FROM the table
con.sql('select name, born from executives')

┌──────────────────────┬─────────────────────┐
│         name         │        born         │
│       varchar        │      timestamp      │
├──────────────────────┼─────────────────────┤
│ Julie_Sweet          │ NULL                │
│ Kumar_Mangalam_Birla │ 1967-06-14 00:00:00 │
│ Shantanu_Narayen     │ 1963-05-27 00:00:00 │
│ Guillaume_Faury      │ 1968-02-22 00:00:00 │
│ Eddie_Wu             │ NULL                │
│ Andy_Jassy           │ 1968-01-13 00:00:00 │
│ Lisa_Su              │ 1969-11-07 00:00:00 │
│ Stephen_Squeri       │ NULL                │
│ Joseph_R._Swedish    │ 1951-05-17 00:00:00 │
│ Tim_Cook             │ 1960-11-01 00:00:00 │
│    ·                 │          ·          │
│    ·                 │          ·          │
│    ·                 │          ·          │
│ Vittorio_Colao       │ 1961-10-03 00:00:00 │
│ Herbert_Diess        │ 1958-10-24 00:00:00 │
│ Robert_Iger          │ 1951-02-10 00:00:00 │
│ Stefano_Pessina      │ 1941-06-04 00:00:00 │
│ Doug_McMill

In [ ]:
# If I use single brackets, I can extract a single column as a Series.
exec_df['name']

,name
0,Julie_Sweet
1,Kumar_Mangalam_Birla
2,Shantanu_Narayen
3,Guillaume_Faury
4,Eddie_Wu
...,...
130,Ann_Sarnoff
131,Jason_Kilar
132,Charles_Scharf
133,John_Mackey


In [ ]:
# We can use SQL over the dataframe OR here
duckdb.sql('select name from exec_df')

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ Julie_Sweet          │
│ Kumar_Mangalam_Birla │
│ Shantanu_Narayen     │
│ Guillaume_Faury      │
│ Eddie_Wu             │
│ Andy_Jassy           │
│ Lisa_Su              │
│ Stephen_Squeri       │
│ Joseph_R._Swedish    │
│ Tim_Cook             │
│    ·                 │
│    ·                 │
│    ·                 │
│ Vittorio_Colao       │
│ Herbert_Diess        │
│ Robert_Iger          │
│ Stefano_Pessina      │
│ Doug_McMillon        │
│ Ann_Sarnoff          │
│ Jason_Kilar          │
│ Charles_Scharf       │
│ John_Mackey          │
│ Rich_Barton          │
├──────────────────────┤
│ 135 rows (20 shown)  │
└──────────────────────┘

In [ ]:
# Notice anything awry?

for person in exec_df['name']:
    print (person)

Julie_Sweet
Kumar_Mangalam_Birla
Shantanu_Narayen
Guillaume_Faury
Eddie_Wu
Andy_Jassy
Lisa_Su
Stephen_Squeri
Joseph_R._Swedish
Tim_Cook
Lakshmi_Niwas_Mittal
John_Stankey
Charles_Woodburn
Tapan_Singhel
Carlos_Torres_Vila
Brian_Moynihan
C.S._Venkatakrishnan
Warren_Buffett
Hubert_Joly
Sunil_Bharti_Mittal
Stephen_A._Schwarzman
Mike_Henry
Oliver_Zipse
Dave_Calhoun
Rich_Lesser
Bob_Dudley
Hock_Tan
Denise_Morrison
Mark_Shuttleworth
Richard_Fairbank
Jim_Umpleby
Evan_Greenberg
Chuck_Robbins
Jane_Fraser
James_Quincey
Brian_L._Roberts
Thomas_Gottstein
Ola_K%C3%A4llenius
Michael_Dell
Ed_Bastian
Christian_Sewing
Tobias_Meyer
Edward_D._Breen
Devin_Wenig
B%C3%B6rje_Ekholm
Darren_Woods
Carmine_Di_Sibio
Mark_Zuckerberg
Frederick_W._Smith
Sergio_Marchionne
Abigail_Johnson
James_Hackett
Terry_Gou
Lachlan_Murdoch
Phebe_Novakovic
H._Lawrence_Culp_Jr.
Mary_T._Barra
Emma_Walmsley
David_M._Solomon
Sundar_Pichai
C_Vijayakumar
Antonio_Neri
Darius_Adamczyk
Noel_Quinn
Arvind_Krishna
Salil_Parekh
Pat_Gelsinger
Jame

In [ ]:
def to_space(x):
  return x.replace('_', ' ')

# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(to_space)

,name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
130,Ann Sarnoff
131,Jason Kilar
132,Charles Scharf
133,John Mackey


In [ ]:
# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(lambda x: x.replace('_', ' '))

,name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
130,Ann Sarnoff
131,Jason Kilar
132,Charles Scharf
133,John Mackey


In [ ]:
# I can also use *apply* to call a function over the rows of a dataframe
exec_df.apply(lambda x: x['name'].replace('_', ' '), axis=1)

,0
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
130,Ann Sarnoff
131,Jason Kilar
132,Charles Scharf
133,John Mackey


In [ ]:
# Let's clean the name by removing underscores...
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))

exec_df

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
4,Eddie_Wu,https://en.wikipedia.org/wiki/Eddie_Wu,NaT,Eddie Wu
...,...,...,...,...
130,Ann_Sarnoff,https://en.wikipedia.org/wiki/Ann_Sarnoff,NaT,Ann Sarnoff
131,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar
132,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24 00:00:00,Charles Scharf
133,John_Mackey,https://en.wikipedia.org/wiki/John_Mackey,NaT,John Mackey


In [ ]:
exec_df.rename(columns={'name': 'old_name'})

,old_name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
4,Eddie_Wu,https://en.wikipedia.org/wiki/Eddie_Wu,NaT,Eddie Wu
...,...,...,...,...
130,Ann_Sarnoff,https://en.wikipedia.org/wiki/Ann_Sarnoff,NaT,Ann Sarnoff
131,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar
132,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24 00:00:00,Charles Scharf
133,John_Mackey,https://en.wikipedia.org/wiki/John_Mackey,NaT,John Mackey


In [ ]:
# We can do the same via SQL.  For the example we'll save the dataframe first...  we'll convert
# types to string first to avoid errors.

duckdb.sql("select name, replace(name, '_', ' ') as clean_name from exec_df")

┌──────────────────────┬──────────────────────┐
│         name         │      clean_name      │
│       varchar        │       varchar        │
├──────────────────────┼──────────────────────┤
│ Julie_Sweet          │ Julie Sweet          │
│ Kumar_Mangalam_Birla │ Kumar Mangalam Birla │
│ Shantanu_Narayen     │ Shantanu Narayen     │
│ Guillaume_Faury      │ Guillaume Faury      │
│ Eddie_Wu             │ Eddie Wu             │
│ Andy_Jassy           │ Andy Jassy           │
│ Lisa_Su              │ Lisa Su              │
│ Stephen_Squeri       │ Stephen Squeri       │
│ Joseph_R._Swedish    │ Joseph R. Swedish    │
│ Tim_Cook             │ Tim Cook             │
│    ·                 │    ·                 │
│    ·                 │    ·                 │
│    ·                 │    ·                 │
│ Vittorio_Colao       │ Vittorio Colao       │
│ Herbert_Diess        │ Herbert Diess        │
│ Robert_Iger          │ Robert Iger          │
│ Stefano_Pessina      │ Stefano Pessina

## 2.1. Selecting a subset of the rows

In [ ]:
# Here's a column

exec_df['clean_name']

,clean_name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
130,Ann Sarnoff
131,Jason Kilar
132,Charles Scharf
133,John Mackey


In [ ]:
# We can apply a test (predicate) to each column, returning a Series of boolean true/false values

exec_df['clean_name'] == 'Julie Sweet'

,clean_name
0,True
1,False
2,False
3,False
4,False
...,...
130,False
131,False
132,False
133,False


In [ ]:
# If we compose these, we'll get only those rows where the boolean condition was True

exec_df[exec_df['clean_name'] == 'Julie Sweet']

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet


SQL lets us use any case, but convention is to capitalize the SQL keywords such as `SELECT`, `FROM`, `WHERE` to aid in readability.  Also, we should use single-quotes for SQL strings, so we'll typically pass the SQL command in with double-quotes.

In [ ]:
duckdb.sql("SELECT * FROM exec_df WHERE clean_name='Julie Sweet'")

┌─────────────┬───────────────────────────────────────────┬───────────┬─────────────┐
│    name     │                   page                    │   born    │ clean_name  │
│   varchar   │                  varchar                  │ timestamp │   varchar   │
├─────────────┼───────────────────────────────────────────┼───────────┼─────────────┤
│ Julie_Sweet │ https://en.wikipedia.org/wiki/Julie_Sweet │ NULL      │ Julie Sweet │
└─────────────┴───────────────────────────────────────────┴───────────┴─────────────┘

In [ ]:
exec_df[exec_df['clean_name'] == 'Julie Sweet'][['page']]

,page
0,https://en.wikipedia.org/wiki/Julie_Sweet



Here we'll use the triple-quote syntax for Python strings, which allows us to pass a multi-line string to SQL...


In [ ]:
duckdb.sql("""SELECT clean_name
            FROM exec_df
            WHERE clean_name='Julie Sweet'""")

┌─────────────┐
│ clean_name  │
│   varchar   │
├─────────────┤
│ Julie Sweet │
└─────────────┘

In [ ]:
import numpy as np

exec_df.dropna(subset=['born'])

,name,page,born,clean_name
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
5,Andy_Jassy,https://en.wikipedia.org/wiki/Andy_Jassy,1968-01-13 00:00:00,Andy Jassy
6,Lisa_Su,https://en.wikipedia.org/wiki/Lisa_Su,1969-11-07 00:00:00,Lisa Su
...,...,...,...,...
127,Robert_Iger,https://en.wikipedia.org/wiki/Robert_Iger,1951-02-10 00:00:00,Robert Iger
128,Stefano_Pessina,https://en.wikipedia.org/wiki/Stefano_Pessina,1941-06-04 00:00:00,Stefano Pessina
129,Doug_McMillon,https://en.wikipedia.org/wiki/Doug_McMillon,1966-10-17 00:00:00,Doug McMillon
131,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar


In [ ]:
duckdb.sql("""SELECT *
            FROM exec_df
            WHERE born IS NOT NULL""")

┌──────────────────────┬──────────────────────────────────────────────────┬─────────────────────┬──────────────────────┐
│         name         │                       page                       │        born         │      clean_name      │
│       varchar        │                     varchar                      │      timestamp      │       varchar        │
├──────────────────────┼──────────────────────────────────────────────────┼─────────────────────┼──────────────────────┤
│ Kumar_Mangalam_Birla │ https://en.wikipedia.org/wiki/Kumar_Mangalam_B…  │ 1967-06-14 00:00:00 │ Kumar Mangalam Birla │
│ Shantanu_Narayen     │ https://en.wikipedia.org/wiki/Shantanu_Narayen   │ 1963-05-27 00:00:00 │ Shantanu Narayen     │
│ Guillaume_Faury      │ https://en.wikipedia.org/wiki/Guillaume_Faury    │ 1968-02-22 00:00:00 │ Guillaume Faury      │
│ Andy_Jassy           │ https://en.wikipedia.org/wiki/Andy_Jassy         │ 1968-01-13 00:00:00 │ Andy Jassy           │
│ Lisa_Su              │ https:/

## 2.2. Joining Data

We start with a simple join between company_ceos_df and exec_df and persist it to the database.  We then check how many companies did not have a match on CEO name.

In [ ]:
exec_df[['clean_name', 'born']]

,clean_name,born
0,Julie Sweet,NaT
1,Kumar Mangalam Birla,1967-06-14 00:00:00
2,Shantanu Narayen,1963-05-27 00:00:00
3,Guillaume Faury,1968-02-22 00:00:00
4,Eddie Wu,NaT
...,...,...
130,Ann Sarnoff,NaT
131,Jason Kilar,1971-04-26 00:00:00
132,Charles Scharf,1965-04-24 00:00:00
133,John Mackey,NaT


In [ ]:
# Remove any duplicate executive entries

exec_df = exec_df.drop_duplicates()

In [ ]:
company_ceos_df[['Executive', 'Company']]

,Executive,Company
0,Julie Sweet,Accenture
1,Kumar Mangalam Birla,Aditya Birla Group
2,Shantanu Narayen,Adobe Systems
3,Guillaume Faury,Airbus
4,Eddie Wu,Alibaba
...,...,...
130,Ann Sarnoff,Warner Brothers
131,Jason Kilar,WarnerMedia
132,Charles Scharf,Wells Fargo
133,John Mackey,Whole Foods Market


In [ ]:
company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'])

,Executive,Company,clean_name,born
0,Julie Sweet,Accenture,Julie Sweet,NaT
1,Kumar Mangalam Birla,Aditya Birla Group,Kumar Mangalam Birla,1967-06-14 00:00:00
2,Shantanu Narayen,Adobe Systems,Shantanu Narayen,1963-05-27 00:00:00
3,Guillaume Faury,Airbus,Guillaume Faury,1968-02-22 00:00:00
4,Eddie Wu,Alibaba,Eddie Wu,NaT
...,...,...,...,...
127,Ann Sarnoff,Warner Brothers,Ann Sarnoff,NaT
128,Jason Kilar,WarnerMedia,Jason Kilar,1971-04-26 00:00:00
129,Charles Scharf,Wells Fargo,Charles Scharf,1965-04-24 00:00:00
130,John Mackey,Whole Foods Market,John Mackey,NaT


We can `JOIN ON` in the `FROM` clause.

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
          """)

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Julie Sweet          │ Accenture                │ NULL                │
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems            │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Eddie Wu             │ Alibaba                  │ NULL                │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ 1969-11-07 00:00:00 │
│ Stephen Squeri       │ American Express         │ NULL                │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple        

Note there is another way you'll sometimes see, in older versions of SQL... Which is to put the join as a `WHERE` condition:

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df, exec_df
            WHERE Executive=clean_name
          """)

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Julie Sweet          │ Accenture                │ NULL                │
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems            │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Eddie Wu             │ Alibaba                  │ NULL                │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ 1969-11-07 00:00:00 │
│ Stephen Squeri       │ American Express         │ NULL                │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple        

OK, let's drop the cases where we don't have a CEO's birthday: these aren't useful!

In [ ]:
# Shall we skip the cases where we don't have the birthday?
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
            WHERE born is not null
          """)

┌──────────────────────┬─────────────────────────────────┬─────────────────────┐
│      Executive       │             Company             │        born         │
│       varchar        │             varchar             │      timestamp      │
├──────────────────────┼─────────────────────────────────┼─────────────────────┤
│ Kumar Mangalam Birla │ Aditya Birla Group              │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems                   │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                          │ 1968-02-22 00:00:00 │
│ Andy Jassy           │ Amazon                          │ 1968-01-13 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices          │ 1969-11-07 00:00:00 │
│ Joseph R. Swedish    │ Anthem                          │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple                           │ 1960-11-01 00:00:00 │
│ Lakshmi Niwas Mittal │ Arcelor Mittal                  │ 1950-06-15 00:00:00 │
│ Charles Woodburn     │ BAE

## 2.4. Finding the misses in the join with OUTER JOINs.

Note that the join above resulted in 174 rows.  However, there are more rows in company_ceos_df so we are missing some companies.  We can see which are missed using a LEFT OUTERJOIN (aka LEFT JOIN); setting "indicator= True" allows us to see which tuples in company_ceos_df failed to find a match (left_only, e.g. row 24 and 172).

In [ ]:
pd.set_option('display.max_rows', 200)
display(company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="left", indicator=True))



,Executive,Company,clean_name,born,_merge
0,Julie Sweet,Accenture,Julie Sweet,NaT,both
1,Kumar Mangalam Birla,Aditya Birla Group,Kumar Mangalam Birla,1967-06-14 00:00:00,both
2,Shantanu Narayen,Adobe Systems,Shantanu Narayen,1963-05-27 00:00:00,both
3,Guillaume Faury,Airbus,Guillaume Faury,1968-02-22 00:00:00,both
4,Eddie Wu,Alibaba,Eddie Wu,NaT,both
5,Andy Jassy,Amazon,Andy Jassy,1968-01-13 00:00:00,both
6,Lisa Su,Advanced Micro Devices,Lisa Su,1969-11-07 00:00:00,both
7,Stephen Squeri,American Express,Stephen Squeri,NaT,both
8,Joseph R. Swedish,Anthem,Joseph R. Swedish,1951-05-17 00:00:00,both
9,Tim Cook,Apple,Tim Cook,1960-11-01 00:00:00,both


In [ ]:
pd.set_option('display.max_rows', 50)
result_df = company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="outer", indicator=True)

result_df[result_df['_merge'] != 'both']


,Executive,Company,clean_name,born,_merge
37,Ola Källenius,Daimler AG,NaN,NaN,left_only
44,Börje Ekholm,Ericsson,NaN,NaN,left_only
98,Michael O'Leary,Ryanair,NaN,NaN,left_only
135,NaN,NaN,Ola K%C3%A4llenius,1969-06-11 00:00:00,right_only
136,NaN,NaN,B%C3%B6rje Ekholm,NaT,right_only
137,NaN,NaN,Michael O%27Leary,NaT,right_only


We can also do this in SQL (there is no indicator but we can test for NULL):

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, clean_name, born
            FROM company_ceos_df FULL JOIN exec_df ON Executive=clean_name
            WHERE clean_name is null or Company is null
          """)

┌─────────────────┬────────────┬────────────────────┬─────────────────────┐
│    Executive    │  Company   │     clean_name     │        born         │
│     varchar     │  varchar   │      varchar       │      timestamp      │
├─────────────────┼────────────┼────────────────────┼─────────────────────┤
│ NULL            │ NULL       │ Ola K%C3%A4llenius │ 1969-06-11 00:00:00 │
│ NULL            │ NULL       │ B%C3%B6rje Ekholm  │ NULL                │
│ NULL            │ NULL       │ Michael O%27Leary  │ NULL                │
│ Michael O'Leary │ Ryanair    │ NULL               │ NULL                │
│ Ola Källenius   │ Daimler AG │ NULL               │ NULL                │
│ Börje Ekholm    │ Ericsson   │ NULL               │ NULL                │
└─────────────────┴────────────┴────────────────────┴─────────────────────┘

## 2.3. Composing Joins

Of course, we can join the results of a join with another table -- representing a *composition*!



Let's join with company data!

In [ ]:
con.sql("""SELECT Executive, Company, born from company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON Company=cd.name
            WHERE born is not null""")

┌───────────┬─────────┬───────────┐
│ Executive │ Company │   born    │
│  varchar  │ varchar │ timestamp │
├───────────┴─────────┴───────────┤
│             0 rows              │
└─────────────────────────────────┘

Hmm, what is wrong here?

In [ ]:
con.sql('SELECT * from company_data')

┌────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ Unnamed: 0 │         name         │ … │     linkedin url     │ current employee e…  │ total employee est…  │
│   int64    │       varchar        │   │       varchar        │        int64         │        int64         │
├────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│    5872184 │ ibm                  │ … │ linkedin.com/compa…  │               274047 │               716906 │
│    4425416 │ tata consultancy s…  │ … │ linkedin.com/compa…  │               190771 │               341369 │
│      21074 │ accenture            │ … │ linkedin.com/compa…  │               190689 │               455768 │
│    2309813 │ us army              │ … │ linkedin.com/compa…  │               162163 │               445958 │
│    1558607 │ ey                   │ … │ linkedin.com/compa…  │               158363 │               428960 │
│

Notice the case for `name`?

In [ ]:
con.sql("""SELECT Executive, Company, born from company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ 1969-11-07 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple                    │ 1960-11-01 00:00:00 │
│ Tim Cook             │ Apple                    │ 1960-11-01 00:00:00 │
│ Tim Cook             │ Apple        

Hmm, there are duplicates!  This is because of fields in the `company_data` table that we don't care about. We can remove the duplicates via `SELECT DISTINCT`.

In [ ]:
con.sql("""SELECT DISTINCT Executive, Company, born from company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ 1969-11-07 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple                    │ 1960-11-01 00:00:00 │
│ Charles Woodburn     │ BAE Systems              │ 1971-03-11 00:00:00 │
│ Oliver Zipse         │ BMW                      │ 1964-02-07 00:00:00 │
│ Bob Dudley           │ BP                       │ 1955-09-14 00:00:00 │
│ Brian Moynihan       │ Bank of Ameri

Can we do all of this in Pandas? Of course!

First, we need to lowercase the company names.

In [ ]:
company_ceos_df['company_lc'] = company_ceos_df['Company'].apply(lambda x: x.lower())

Notice this is slower than DuckDB?

In [ ]:
company_ceos_df.merge(exec_df.dropna(),
                      left_on=['Executive'],
                      right_on=['clean_name']).\
                      merge(company_data_df,
                            left_on='company_lc',
                            right_on='name')[['Executive','Company','born']].drop_duplicates().sort_values('Company')

,Executive,Company,born
0,Kumar Mangalam Birla,Aditya Birla Group,1967-06-14 00:00:00
5,Lisa Su,Advanced Micro Devices,1969-11-07 00:00:00
1,Guillaume Faury,Airbus,1968-02-22 00:00:00
3,Andy Jassy,Amazon,1968-01-13 00:00:00
6,Joseph R. Swedish,Anthem,1951-05-17 00:00:00
...,...,...,...
136,Vittorio Colao,Vodafone,1961-10-03 00:00:00
139,Stefano Pessina,Walgreens Boots Alliance,1941-06-04 00:00:00
140,Doug McMillon,Walmart,1966-10-17 00:00:00
138,Robert Iger,Walt Disney Company,1951-02-10 00:00:00


## Section 2 Exercises

Hmm, there seem to be quite a few Roberts and Bobs. (Maybe also Roberta?)

Using either Pandas operations like `merge` and the various bracket notations (but **not** the `loc`/`iloc` commands if you know these): write a query that takes the `company_ceos_df` and returns all companies that are overseen by someone with names `Bob`, `Robert`, or `Roberta`.

First, define a function that *robustly* does this:

In [ ]:
def bob(name: str) -> bool:
  first_name = name.split(' ')[0].lower().strip()
  return first_name == 'bob' or \
    first_name == 'robert' or \
    first_name == 'roberta'

In [ ]:
from dill.source import getsource
grader.grade('bob_test', getsource(bob))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [ ]:
bob_co_df = company_ceos_df[company_ceos_df['Executive'].apply(bob)][['Executive','Company']]
bob_co_df

,Executive,Company
25,Bob Dudley,BP
73,Bob Sternfels,McKinsey & Company
79,Robert James Thomson,News Corp
108,Robert E. Jordan,Southwest Airlines
124,Robert Bakish,ViacomCBS
127,Robert Iger,Walt Disney Company


In [ ]:
grader.grade(test_case_id='bobs_your_uncle_or_aunt', answer=bob_co_df)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.
